<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introcción

#2. Armado del ambiente

In [ ]:
url_archivo_1 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2014.txt?raw=true"
url_archivo_2 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2015.txt?raw=true"
url_archivo_3 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2016.txt?raw=true"

!wget {url_archivo_1} -O "2014.txt"
!wget {url_archivo_2} -O "2015.txt"
!wget {url_archivo_3} -O "2016.txt"

In [ ]:
!ls
!cat 2015.txt

#3. Desarrollo

In [81]:
code = """
// Procesamiento de archivos con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <fstream>
#include <string>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i;
  int cant_files;
  TIEMPO_INI( HTH_TOTAL )

  /*if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }*/

  int cantidad_N = atoi(argv[1]);
  int number_of_lines[cantidad_N] = {0, 0, 0};
   TIEMPO_INI( HTH_AXPY_OMP )

    cant_files = atoi(argv[1]);

    #pragma omp parallel for
    for(i=0; i < cant_files; i++){
      number_of_lines[i] = 0;
      std::string line;
      std::ifstream myfile(argv[2 + i]);
      while (std::getline(myfile, line)){
        number_of_lines[i]++;
      }
      std::cout << "Nombre de archivo: " << argv[2 + i];
      std::cout << " " << std::endl;
      std::cout << "Lineas de texto de archivo: " << number_of_lines[i];
      std::cout << " " << std::endl;
    }
    
    TIEMPO_FIN( HTH_AXPY_OMP )



  TIEMPO_FIN( HTH_AXPY_OMP )

  //-------------CIERRO_ARCHIVOS------------------
  // --------------------------------------------

}
"""
text_file = open("code_axpy.cpp", "w")
text_file.write(code)
text_file.close()

In [79]:
!g++ -o axpy -fopenmp code_axpy.cpp

In [80]:
%env OMP_NUM_THREADS=3
!./axpy 3 "2015.txt" "2016.txt" "2014.txt"

env: OMP_NUM_THREADS=3
Name of file: 2014.txt 
Number of lines in text file: 30326 
Name of file: 2015.txt 
Number of lines in text file: 44724 
Name of file: 2016.txt 
Number of lines in text file: 43704 


#4. Tabla de pasos

#5. Conclusiones